# Пару слов о блокноте

__Задача блокнота__: проектирование и обучение модели машинного обучения для решения задачи NER (Named Entity Recognition).

__P. S.__: блокнот является второй частью (последней) производственной практики по учебному курсу дополнительного образования. Основной используемой библиотекой будет `HuggingFace`, которая предосатвлет API для использования моделей популярных, а также пользовательских моделей трансформеров. Я буду использовать найденную модель и дообучать ее для решению моей задачи.

In [1]:
! pip install datasets seqeval -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [2]:
import pandas as pd
import datasets
import numpy as np
from sklearn.model_selection import train_test_split
from transformers import BertTokenizerFast # try bert for token classification
from transformers import AutoModelForTokenClassification
from transformers import DataCollatorForTokenClassification
from transformers import TrainingArguments, Trainer
from transformers import pipeline
import json
from tokenizers.pre_tokenizers import Whitespace
import re
import os
import shutil
import torch

In [3]:
labels2ind = {
    'O': 0,
    'B-Knowledge': 1,
    'I-Knowledge': 2,
    'B-SoftSkills': 3,
    'I-SoftSkills': 4,
    'B-Tool': 5,
    'I-Tool': 6,
    'B-ProgrammingLanguage': 7,
    'I-ProgrammingLanguage': 8,
    'B-Method': 9,
    'I-Method': 10,
    'B-Technology': 11,
    'I-Technology': 12,
}
ind2labels = {v: k for k, v in labels2ind.items()}

In [4]:
labels_list = [key for key, value in list(labels2ind.items())]
labels_list

['O',
 'B-Knowledge',
 'I-Knowledge',
 'B-SoftSkills',
 'I-SoftSkills',
 'B-Tool',
 'I-Tool',
 'B-ProgrammingLanguage',
 'I-ProgrammingLanguage',
 'B-Method',
 'I-Method',
 'B-Technology',
 'I-Technology']

## 1. Загрузка предобработанных данных

Данные были обработаны и объединены в один файл `data_clean.json`. Загрузим этот файл.

In [20]:
df = pd.read_json('data_clean.json')
df.head(3)

,tokens,labels
0,"[101144061, тьютор, (, помощник, ), обязанност...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
1,"[102098163, психолог, обязанности, :, организа...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
2,"[101516171, педагог, -, психолог, црвсп, -, ро...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."


## 2. Подготовка данных и модели для обучения

#### 2.1. Объединение данных в датасет библиотеки HuggingFace

In [21]:
train, test = train_test_split(df, test_size=0.2)
val, test = train_test_split(test, test_size=0.5)

df_dict = dict()
df_dict['train'] = train
df_dict['validate'] = val
df_dict['test'] = test

In [22]:
dataset = datasets.DatasetDict({
    'train': datasets.Dataset.from_pandas(df_dict['train']),
    'validation': datasets.Dataset.from_pandas(df_dict['validate']),
    'test': datasets.Dataset.from_pandas(df_dict['test'])})

dataset

DatasetDict({
    train: Dataset({
        features: ['tokens', 'labels', '__index_level_0__'],
        num_rows: 9103
    })
    validation: Dataset({
        features: ['tokens', 'labels', '__index_level_0__'],
        num_rows: 1138
    })
    test: Dataset({
        features: ['tokens', 'labels', '__index_level_0__'],
        num_rows: 1138
    })
})

In [23]:
# example
print(dataset['train'][0]['tokens'])
print(dataset['train'][0]['labels'])

[':', 'методы', 'оценки', 'объемов', 'и', 'сроков', 'работ', 'по', 'реализации', 'проектов', 'в', 'продуктивную', 'среду', ';', 'экспертное', 'и', 'профессиональное', 'знание', ':', 'os', 'linux', '(', 'centos', ',', 'debian', '),', 'систем', 'виртуализации', '(', 'esxi', ',', 'hyper', '-', 'v', ',', 'proxmox', '),', 'субд', '(', 'psql', ',', 'mongo', ',', 'riak', '),', 'протоколов', 'передачи', 'данных', '(', 'tcp', ',', 'udp', '),', 'языков', 'программирования', '(', 'python', ',', 'bash', ')', 'для', 'написания', 'скриптов', 'автоматизации', ',', 'систем', 'контейнеризации', 'и', 'управления', 'контейнерами', '(', 'docker', ',', 'docker', '-', 'compose', ');', 'желателен', 'опыт', 'работы', 'с', 'системами', 'и', 'по', ',', 'работающими', 'в', 'закрытых', 'контурах', '.', 'centos', '1', '.', 'gitlab', ',', 'team', 'city', ',', 'ansible', ',', 'bash', '/', 'sh', '2', '.', 'postgresql', ',', 'mongodb', ',', 'oracle', '3', '.', 'hadoop', ',', 'apache', 'spark', ',', 'apache', 'ignite',

#### 2.2. Импорт токенайзера

In [24]:
# лучший из токенайзеров для русского языка, который я смог найти на HuggingFace hub
model_name = 'KoichiYasuoka/bert-base-russian-upos'

tokenizer = BertTokenizerFast.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/236 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.65M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.62M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Посмотрим на результат токенизации одного из экземпляров датасета

In [25]:
example_text = dataset['train'][0]['tokens']
tokenized_input = tokenizer(example_text, is_split_into_words=True,
                            padding=True, truncation=True,
                            max_length=512, add_special_tokens=False)

tokens = tokenizer.convert_ids_to_tokens(tokenized_input['input_ids'])

# comapare our tokens and tokenizer's tokens
i = 0
for token in tokens:
    if i >= len(tokens) or i >= len(example_text):
        continue

    if example_text[i].startswith(token):
        print('======================================================')
        space = (30 - len(token)) * ' '
        print(f'{token}{space}{example_text[i]}')
        i += 1
    else:
        print(token)
    #print(token, '\t\t', (example_text[i] if i < len(example_text) else ''))

:                             :
методы                        методы
оценки                        оценки
объемов                       объемов
и                             и
сроков                        сроков
работ                         работ
по                            по
реализации                    реализации
проектов                      проектов
в                             в
продукт                       продуктивную
##ивную
среду                         среду
;                             ;
эксперт                       экспертное
##ное
и                             и
профессиональное              профессиональное
знание                        знание
:                             :
os                            os
li                            linux
##nu
##x
(                             (
cent                          centos
##os
,                             ,
de                            debian
##bia
##n
)                             ),
,
систем                    

Можно заметить, что tokenizer разбил токены не идеально: многие слова дополнительно доразибваются на более маленькие токены с символами ## (происходит, потому что tokenizer не знает этих слов).

In [26]:
len(dataset['train'][0]['tokens']), len(tokenized_input['input_ids'])

(155, 238)

Выше четко видна разница в количестве наших токенов и токенов полученных токенайзером.

#### 2.3. Токенизация датасета

Проведем токнизацию датасета. Для этого определим функцию токнизации, котрая будет работать согласно алгоритму ниже

Сам алгоритм:<br>
нужно решить проблему разбиения токенайзером уже готовых токенов: будем брать марку токена и распределять ее на все остальные рабитые токены. <br>
Например, если токенайзер разбил слово 'Станок' на ['Стан', '##ок'], и слово имело марку "B-Tool", то сопоставим обоим полученным словам марку "B-Tool".

In [27]:
def tokenize_and_align_labels(example):
    tokenized_input = tokenizer(example['tokens'], truncation=True, padding=True,
                                max_length=512, is_split_into_words=True)
                                # max_length = 512 because BERT model can't procces more than 512 tokens
    labels = []

    for i, label in enumerate(example['labels']):
        word_ids = tokenized_input.word_ids(batch_index=i)

        previous_word_idx = None

        label_ids = []

        for word_idx in word_ids:
            if word_idx is None:
                # igonre word during training
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                # add new label
                label_ids.append(labels2ind[label[word_idx]])
            else:
                # repeat label
                label_ids.append(labels2ind[label[word_idx]])
            previous_word_idx = word_idx
        labels.append(label_ids)
    tokenized_input['labels'] = labels
    return tokenized_input

In [28]:
# check example
tokenize_and_align_labels(dataset['train'][2:3])

{'input_ids': [[101, 26661, 156, 130, 22310, 33053, 9449, 37789, 33098, 28552, 11022, 158, 130, 14712, 21825, 874, 132, 20859, 20677, 132, 24893, 851, 77508, 13763, 871, 7512, 54908, 10937, 263, 128, 47634, 851, 12058, 33199, 128, 8871, 2630, 75527, 128, 85424, 1662, 851, 44328, 51895, 128, 22321, 871, 7512, 132, 3455, 28267, 17683, 42100, 851, 10887, 1638, 132, 102]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'labels': [[-100, 0, 0, 0, 9, 10, 10, 10, 10, 10, 10, 0, 0, 0, 5, 5, 0, 9, 10, 0, 0, 0, 0, 0, 5, 5, 6, 6, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 11, 11, 0, 0, 0, 0, 5, 0, 5, 5, 0, -100]]}

Таким образом, к токенизированному выводу мы получили дополнительный ключ "labels"

In [29]:
tokenized_dataset = dataset.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/9103 [00:00<?, ? examples/s]

Map:   0%|          | 0/1138 [00:00<?, ? examples/s]

Map:   0%|          | 0/1138 [00:00<?, ? examples/s]

#### 2.4. Импорт модели и загрузка метрик

In [30]:
model = AutoModelForTokenClassification.from_pretrained(model_name, num_labels=13, ignore_mismatched_sizes=True)

config.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/709M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at KoichiYasuoka/bert-base-russian-upos and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([89, 768]) in the checkpoint and torch.Size([13, 768]) in the model instantiated
- classifier.bias: found shape torch.Size([89]) in the checkpoint and torch.Size([13]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [31]:
metric = datasets.load_metric('seqeval')

<ipython-input-31-8729a7e2f3c1>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = datasets.load_metric('seqeval')


The repository for seqeval contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/seqeval.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


In [32]:
# example
metric.compute(predictions=[labels], references=[labels])

{'Method': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 2},
 'Technology': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 4},
 'Tool': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'overall_precision': 1.0,
 'overall_recall': 1.0,
 'overall_f1': 1.0,
 'overall_accuracy': 1.0}

In [33]:
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [ind2labels[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [ind2labels[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

## 3. Дообучение импортированной модели

In [34]:
data_collator = DataCollatorForTokenClassification(tokenizer)

In [35]:
training_args = TrainingArguments(
    output_dir="test_NER",
    eval_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=5,
    weight_decay=0.05,
    save_strategy='epoch'
)

In [36]:
trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [37]:
trainer.train()

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.559686,0.587270,0.533022,0.558832,0.835031
2,0.649200,0.514697,0.582727,0.584412,0.583568,0.841643
3,0.649200,0.502450,0.575012,0.599308,0.586908,0.842926
4,0.495600,0.498069,0.564700,0.618772,0.590501,0.841855
5,0.495600,0.497309,0.561458,0.621124,0.589786,0.841558


TrainOutput(global_step=1425, training_loss=0.5390753173828124, metrics={'train_runtime': 4942.8949, 'train_samples_per_second': 9.208, 'train_steps_per_second': 0.288, 'total_flos': 1.189410463781376e+16, 'train_loss': 0.5390753173828124, 'epoch': 5.0})

In [38]:
model.save_pretrained('ner_model')
tokenizer.save_pretrained('ner_tokenizer')

('ner_tokenizer/tokenizer_config.json',
 'ner_tokenizer/special_tokens_map.json',
 'ner_tokenizer/vocab.txt',
 'ner_tokenizer/added_tokens.json',
 'ner_tokenizer/tokenizer.json')

Я тренировал модель на 5-ти эпохах. На полученные значения метрик Recall, Precision, F1 не стоит обращать внимания, т.к. они явно некорректные. Показателем обучения является ошибка тренировки и валиадции (видна выше). Тренировка даже 5 эпопх заняла около 1 часа 25 мин на Google Colab T4 (у меня нет платной подписки, поэтому я не буду ждать дольше). При желании можно будет запустить обучение вновь с большим числом эпох, если имеется подписка на более мощный процессор, или есть желание ждать больше времени.

## 4. Оценка результатов

Результаты обучения видны в предыдущем пункте. Минимум ошибки валидации после обучения составил 0.5.<br>

Небольшую точность также можно объяснить следующими вещами: <br>
 1) Был выбран неидеальный токенайзер (однако лучший из мной найденных); <br>
 2) Не очень большим размером пакета (32), т.к. GoogleColab T4 не может обрабатывать более большой пакет.

Укажем модели, какие метки она предсказывает:

In [39]:
config = json.load(open('ner_model/config.json'))

config['id2label'] = ind2labels
config['label2id'] = labels2ind

json.dump(config, open('ner_model/config.json', 'w'))

Загрузим модель и токенайзер:

In [40]:
# load our model
model_fine_tuned = AutoModelForTokenClassification.from_pretrained('ner_model')

# load our tokenizer
tokenizer_fine_tuned = BertTokenizerFast.from_pretrained('ner_tokenizer')

#### 4.1. Вычисление F1 меры для двух доп. файликов

Попробуем написать функцию для высчитывания метрики F1 с использованием нашей модели.<br>
Высчитывать метрику будем по стандартной формуле, предварительно получив значения TP, FP, TN, FN.<br>
Чтобы иметь возможность сравнить наши собственные токены и мектки с токенами, полученными после HuggingFace токенайзера (с определенными для них метками), потребуются доп. функции.

Сохраним данные с доп. файликов в датасеты

In [41]:
df1 = pd.read_json('all_extended_proc_annot.jsonl', lines=True)
df2 = pd.read_json('all_extended_proc_annot_squeezed.jsonl', lines=True)
print(df1.shape[0])
print(df2.shape[0])
print(df1.columns)
print(df2.columns)

5184
5071
Index(['id', 'text', 'cats', 'entities', 'Comments', 'kind', 'annot', 'meta'], dtype='object')
Index(['id', 'text', 'cats', 'entities', 'Comments', 'kind', 'annot', 'meta'], dtype='object')


Чтобы занового не вызывать функцию text_to_tokens, получим заранее обработанный вариант этих датасетов через срезы из ранее полученного полного очищенного датасета

In [42]:
df2_clean = df.tail(df2.shape[0])
df1_clean = df.iloc[df.shape[0] - df2.shape[0] - df1.shape[0] : df.shape[0] - df2.shape[0]]

df1_clean = df1_clean.reset_index()
df2_clean = df2_clean.reset_index()

# check synchronization
print(df1.loc[0]['text'][:50])
print(df1_clean.loc[0]['tokens'][:10])
print(df2.loc[0]['text'][:50])
print(df2_clean.loc[0]['tokens'][:10])

Разработчик Flutter Developer. Санкт-Петербургский
['разработчик', 'flutter', 'developer', '.', 'санкт', '-', 'петербургский', 'информационно', '-', 'аналитический']
Разработчик Flutter Developer. Чем необходимо буде
['разработчик', 'flutter', 'developer', '.', 'чем', 'необходимо', 'будет', 'заниматься', ':', 'разработкой']


Значения совпали, следовтаельно, мы получили токенизированные данные именно для наших файликов

Функция ниже преборазует предсказания модели так, что слова, которые были дополнительно разбиты токенайзером, сливаются в исходные с сохранением метки, которую имело первое слово в разбзиении

In [43]:
# merge splitted tokens with saving its labels
def merge_spitted_tokens(prediction):
    tokens = prediction[0]
    labels = prediction[1]

    new_tokens = []
    new_labels = []

    current_label = labels[0]
    current_token = tokens[0]
    for i in range(len(tokens) - 1):
        token = tokens[i]
        label = labels[i]

        if tokens[i + 1].startswith("##"):
           current_token += tokens[i + 1][2:]
        else:
            new_tokens.append(current_token)
            new_labels.append(current_label)

            current_label = labels[i + 1]
            current_token = tokens[i + 1]

    if tokens[-1].startswith("##"):
        new_tokens.append(current_token)
        new_labels.append(current_label)
    else:
        new_tokens.append(tokens[-1])
        new_labels.append(labels[-1])

    return [new_tokens, new_labels]

# Example
prediction = [
    ["Привет", ",", "я", "програ", "##ммист", ".", "Изучаю", "БД", "и", "их", "струк", "##тур", "##ные"],
    ["O", "O", "B-Person", "B-Profession", "B-Profession", "O", "O", "B-Tool", "O", "O", "B-Adjective", "B-Adjective", "B-Adjective"]
]

new_prediction = merge_spitted_tokens(prediction)
print(new_prediction[0])
print(new_prediction[1])

['Привет', ',', 'я', 'программист', '.', 'Изучаю', 'БД', 'и', 'их', 'структурные']
['O', 'O', 'B-Person', 'B-Profession', 'O', 'O', 'B-Tool', 'O', 'O', 'B-Adjective']


Функция ниже также решает проблему токенайзера. Есть ситации, когда в наших токенах есть, например, токен: "; ". Тогда он будет разбит токенайзером на два отдельных токена ";" и " " без добавления символов "##". Такие ситации решает фукнция ниже, возвращая эти токены к исходному с добавлением ему метки "O"

In [44]:
def sinchronize_prediction_by_tokens(prediction, tokens):
    pred_tokens = prediction[0]
    pred_labels = prediction[1]
    new_tokens = []
    new_labels = []

    ind_real = ind_pred = 0
    while (ind_real < len(tokens)) and (ind_pred < len(pred_tokens)):
        real_token = tokens[ind_real]
        pred_token = pred_tokens[ind_pred]
        pred_label = pred_labels[ind_pred]

        if real_token == pred_token:
            new_tokens.append(real_token)
            new_labels.append(pred_label)
            ind_real += 1
            ind_pred += 1
        else:
            new_tokens.append(real_token)
            new_labels.append("O") # assume that splitted labels are not even words
            ind_real += 1
            ind_pred += 1

            if ind_real < len(tokens):
                while (ind_pred < len(pred_tokens)) and (not tokens[ind_real].startswith(pred_tokens[ind_pred])):
                    ind_pred += 1

    return [new_tokens, new_labels]

Функция ниже получает предсказания. Она подает токенайзеру текст пачками так, чтобы не получилось больше 512 токенов (т.к. модель BERT не подходит для более, чем 512 токенов). В итоге вычисляются предсказания для каждой отдельной пачки токенов и в результате они конкатенируются к общему массиву предсказаний. По итогу мы получаем массив предсказаний, где для каждого текста имеется массив его токенов и соответствующий массив меток

In [45]:
def get_predictions(model, tokenzier, df):
    preds = [ [[], []] for i in range(df.shape[0]) ]
    model_id2label = model.config.id2label

    for i in range(df.shape[0]):
        start_index = 0
        current_prediction_full = [[], []]
        df_tokens = df["tokens"][i]

        while True:

            df_tokens_current = df["tokens"][i][start_index:]
            tokenized_input = tokenizer(df_tokens_current, is_split_into_words=True,
                                        padding=True, truncation=True, max_length=512,
                                        add_special_tokens=False, return_tensors="pt")
            model.to('cuda')
            with torch.no_grad():
                outputs = model(tokenized_input["input_ids"].cuda(), tokenized_input["attention_mask"].cuda())

                logits = outputs.logits
                predictions = torch.argmax(logits, dim=2).cuda()
                #label_inds = np.argmax(outputs[0].to('cpu').numpy(), axis=2)

            tokenizer_tokens = tokenizer.convert_ids_to_tokens(tokenized_input['input_ids'][0])
            predicted_labels = [model_id2label[prediction.item()] for prediction in predictions[0]]

            current_prediction = [tokenizer_tokens, predicted_labels]

            current_prediction = merge_spitted_tokens(current_prediction)
            current_prediction = sinchronize_prediction_by_tokens(current_prediction, df_tokens_current)

            current_prediction_full[0] += current_prediction[0]
            current_prediction_full[1] += current_prediction[1]

            start_index = len(current_prediction_full[0])
            if start_index >= len(df_tokens):
                break

        # merge results to one prediction
        preds[i][0] += current_prediction_full[0]
        preds[i][1] += current_prediction_full[1]

        if (i + 1) % 100 == 0:
            print(f"{i + 1} / {df.shape[0]}")

    return preds

In [46]:
predictions_1 = get_predictions(model_fine_tuned, tokenizer_fine_tuned, df1_clean)

100 / 5184
200 / 5184
300 / 5184
400 / 5184
500 / 5184
600 / 5184
700 / 5184
800 / 5184
900 / 5184
1000 / 5184
1100 / 5184
1200 / 5184
1300 / 5184
1400 / 5184
1500 / 5184
1600 / 5184
1700 / 5184
1800 / 5184
1900 / 5184
2000 / 5184
2100 / 5184
2200 / 5184
2300 / 5184
2400 / 5184
2500 / 5184
2600 / 5184
2700 / 5184
2800 / 5184
2900 / 5184
3000 / 5184
3100 / 5184
3200 / 5184
3300 / 5184
3400 / 5184
3500 / 5184
3600 / 5184
3700 / 5184
3800 / 5184
3900 / 5184
4000 / 5184
4100 / 5184
4200 / 5184
4300 / 5184
4400 / 5184
4500 / 5184
4600 / 5184
4700 / 5184
4800 / 5184
4900 / 5184
5000 / 5184
5100 / 5184


In [50]:
predictions_2 = get_predictions(model_fine_tuned, tokenizer_fine_tuned, df2_clean)

100 / 5071
200 / 5071
300 / 5071
400 / 5071
500 / 5071
600 / 5071
700 / 5071
800 / 5071
900 / 5071
1000 / 5071
1100 / 5071
1200 / 5071
1300 / 5071
1400 / 5071
1500 / 5071
1600 / 5071
1700 / 5071
1800 / 5071
1900 / 5071
2000 / 5071
2100 / 5071
2200 / 5071
2300 / 5071
2400 / 5071
2500 / 5071
2600 / 5071
2700 / 5071
2800 / 5071
2900 / 5071
3000 / 5071
3100 / 5071
3200 / 5071
3300 / 5071
3400 / 5071
3500 / 5071
3600 / 5071
3700 / 5071
3800 / 5071
3900 / 5071
4000 / 5071
4100 / 5071
4200 / 5071
4300 / 5071
4400 / 5071
4500 / 5071
4600 / 5071
4700 / 5071
4800 / 5071
4900 / 5071
5000 / 5071


Проверим, действительно ли токены в предсказаниях соответствуют токенам в нашем датасете

In [47]:
# if there is any print so there are issues in methods above
for i in range(len(predictions_1)):
    if len(predictions_1[i][0]) != len(df1_clean.loc[i]['tokens']):
        print('!!!')
        print(i)
        break
    if len(predictions_1[i][0]) != len(predictions_1[i][1]):
        print('!!!!')
        print(i)
        break
    for real_token, pred_token in zip(predictions_1[i][0], df1_clean.loc[i]['tokens']):
        if real_token != pred_token:
            print('!!!')
            print(i)
            break

Функция ничего не вывела, а значит все методы обработки выше - правильные

Функции ниже вычисляют метрики для полученных предсказаний и исходных обработанных данных

In [48]:
def calculate_Recall_Precision_F1(TP, FP, TN, FN):
    if TP + FP > 0:
        precision = TP / (TP + FP)
        recall = TP / (TP + FN)
        F1 = 2 * precision * recall / (precision + recall)
        return [recall, precision, F1]
    return [0, 0, 0]

def calculate_metrics_overall(df, predictions, labels_list):
    TP = FP = TN = FN = 0

    individual_metrics = dict()
    for label in labels_list:
        individual_metrics[label] = [0, 0] # TP FP

    for i in range(len(predictions)):
        sample = df.loc[i]
        predicted_labels = predictions[i][1]

        if len(sample['labels']) != len(predicted_labels):
            print("Incorrect data processing was discovered!")
            print(f"{len(sample['labels'])} != {len(predicted_labels)}")
            return -1

        for label_real, label_pred in zip(sample['labels'], predicted_labels):
            if label_real == label_pred:
                if label_real == "O":
                    TN += 1
                else:
                    TP += 1
                individual_metrics[label_real][0] += 1
            else:
                if label_real == "O":
                    FN += 1
                else:
                    FP += 1
                individual_metrics[label_real][1] += 1

        if (i + 1) % 1000 == 0:
            print(f"{i + 1} / {df.shape[0]}")

    for label in labels_list:
        sample = individual_metrics[label]
        if (sample[0] + sample[1]) > 0:
            individual_metrics[label] = sample[0] / (sample[0] + sample[1]) # accuracy
        else:
            print(f"Class {label} wasn't met")
            individual_metrics[label] = 0

    print(TP, FP, TN, FN)
    precision = TP / (TP + FP) if TP > 0 else 0
    recall = TP / (TP + FN) if TP > 0 else 0
    F1 = 2 * precision * recall / (precision + recall) if precision > 0 else 0

    return [precision, recall, F1], individual_metrics

In [49]:
result1 = calculate_metrics_overall(df1_clean, predictions_1, labels_list)
result1

1000 / 5184
2000 / 5184
3000 / 5184
4000 / 5184
5000 / 5184
Class B-Knowledge wasn't met
Class I-Knowledge wasn't met
30834 222259 2337085 7491


([0.1218287348919172, 0.8045401174168297, 0.21161355853104474],
 {'O': 0.9968049660151772,
  'B-Knowledge': 0,
  'I-Knowledge': 0,
  'B-SoftSkills': 0.09773598911295311,
  'I-SoftSkills': 0.019033605468008353,
  'B-Tool': 0.5337399697983596,
  'I-Tool': 0.21167647770341103,
  'B-ProgrammingLanguage': 0.3559485769487999,
  'I-ProgrammingLanguage': 0.0008853474988933156,
  'B-Method': 0.00807627593942793,
  'I-Method': 0.0065024402253406905,
  'B-Technology': 0.03793443210318882,
  'I-Technology': 0.0019982871824150727})

In [51]:
result2 = calculate_metrics_overall(df2_clean, predictions_2, labels_list)
result2

1000 / 5071
2000 / 5071
3000 / 5071
4000 / 5071
5000 / 5071
Class B-Knowledge wasn't met
Class I-Knowledge wasn't met
36661 218249 627372 3233


([0.1438193872347103, 0.918960244648318, 0.2487144000759827],
 {'O': 0.994873177345565,
  'B-Knowledge': 0,
  'I-Knowledge': 0,
  'B-SoftSkills': 0.13429106808244848,
  'I-SoftSkills': 0.03837417234546421,
  'B-Tool': 0.5726345828007491,
  'I-Tool': 0.23626717908016207,
  'B-ProgrammingLanguage': 0.48421679571173315,
  'I-ProgrammingLanguage': 0.0070859167404783,
  'B-Method': 0.018032159264931086,
  'I-Method': 0.020738195986425907,
  'B-Technology': 0.058702885009799005,
  'I-Technology': 0.0032455824017309774})

Таким образом, для первого файлика F1 = 0.2116, а для второго - 0.2487.

Однако заметно, что модель очень плохо распознает `I-` классы. Наверное, стоит подумать о другом подходе к обучению

#### 4.2. Проверка модели на реальных текстах

Тестировать модель я буду с использованием метода pipeline, т.к. я точно знаю, что мои текста после токенайзера не будут содержать более, чем 512 токенов. В этом случае использование метода pipeline - самый простой способ

Попробуем для теста подать на вход модели один из тренировочных текстов. Из-за оссобенностей модели трансформера он будет рассматриваться как новый.

In [52]:
recognizer = pipeline("ner", model=model_fine_tuned, tokenizer=tokenizer_fine_tuned)

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [53]:
# test the model

example = "102651095\nДоцент кафедры иностранных языков\nУважаемые соискатели!\n \nНа общеуниверситетскую кафедру иностранных языков требуется доцент.\n \n \n \nОбязанности:\n \n \nПроведение всех форм занятий с обучающимися кафедры иностранных языков;\n \nРуководство курсовыми и выпускными квалификационными работами обучающихся; руководство педагогической и научной практикой;\n \nРазработка учебных планов и методических материалов (РПД, программ практик) для обеспечения учебного процесса по преподаваемым дисциплинам;\n \nОценка эффективность обучения предмету обучающихся;\n \nУчастие в научных мероприятиях кафедры;\n \nУчастие в научной и воспитательной работе с обучающимися.\n \n \nТребования:\n \n \nВысшее образование;\n \nНаличие ученой степени кандидата наук\n;\n \nНаличие обязательных курсов повышения квалификации;\n \nСтаж научно-педагогической работы не менее 3 лет;\n \nЗнание методики преподавания английского языка;\n \nПодготовка и публикация научных статей в рецензируемых изданиях ВАК и\/или международных изданиях, регулярное участие в научных конференциях;\n \nУмение работать с компьютерными программами: Word, Excel, Power Point;\n \nРазвитые коммуникативные навыки, трудолюбие.\n \n \nУсловия:\n \n \nРабота в одном из ведущих педагогических университетов России (\nТОП-100 RAEX\n);\n \nОформление в соответствии с ТК РФ;\n \nЗаработная плата состоит из основной части + доплаты в рамках эффективного контракта;\n \nГрафик работы в соответствии с расписанием занятий.\n \nАнглийский язык,Организация учебного процесса,Научная деятельность,Дистанционное обучение\n"

results = recognizer(example)

for result in results:
    print(result['entity'], result['word'], result['score'])

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


I-Knowledge преподавания 0.4974931
I-Knowledge английского 0.5063019
I-Knowledge языка 0.5535979
B-Tool Word 0.6044043
B-Tool Excel 0.5858517
B-Tool Power 0.69531226
I-Tool Point 0.6889092
B-SoftSkills Развит 0.53862053
B-SoftSkills ##ые 0.49957955
B-SoftSkills коммуника 0.6755258
B-SoftSkills ##тив 0.8492422
B-SoftSkills ##ные 0.8340413
I-SoftSkills навыки 0.7392947
B-SoftSkills трудолюб 0.8257472
B-SoftSkills ##ие 0.8529555
B-Knowledge Английский 0.45878327
I-Knowledge язык 0.5053182
B-Knowledge Организация 0.4955746
I-Knowledge учебного 0.526317
I-Knowledge процесса 0.522805
B-Knowledge Научная 0.4811884
I-Knowledge деятельность 0.52109134
B-Knowledge ##ционное 0.51979804
I-Knowledge обучение 0.5785401


Стоит отметить, что результат весьма впечатляющий. Попробуем подать небольшой текст

In [54]:
# test the model

example = "Мы - развивающаяся IT-компания. Ищем ответственного человека, способного работать в команде, учавствовать в разработке приложений на языке C++. Требования: -уверенное владение английским языком (C2); -разработка и поддержание готовых проектов; -хороший внешний вид. Оплата: 500 тыс. рублей"

results = recognizer(example)

for result in results:
    print(result['entity'], result['word'], result['score'])

I-SoftSkills команде 0.5808319
B-ProgrammingLanguage C 0.88274884
I-ProgrammingLanguage + 0.96050084
I-ProgrammingLanguage + 0.96728146
B-SoftSkills английским 0.52719045
I-SoftSkills языком 0.51900136
I-SoftSkills вид 0.6054899


Результат, к сожалению, не очень хороший. Однако круто, что модель смогла хоть как-то отработать с маленьким текстов, учитывая, что тренировка проводилась только на больших текстах

Попробую подать на вход более большой текст

In [55]:
# test the model

# hh.ru
example = "Наша компания является независимым разработчиком программного обеспечения в области систем управления версиями (version control) и у нас есть вакансия Junior Software Engineer. Данная вакансия идеальна для недавних выпускников и студентов профильных специальностей. Мы не требуем многолетний опыт работы, готовы помогать осваивать навыки системного программирования и предлагаем высокую заработную плату и комфортные условия работы. Отличный вариант для талантливых и увлеченных.\n\nНаши требования:\n\nПродвинутое владение хотя бы одним современным языком общего назначения (С/C++, C# или Java).\nФундаментальные знания в области программирования (структуры данных, алгоритмы и т.п.).\nОбщее знакомство с языком C/C++.\nТехнический английский.\nБудет плюсом:\n\nПрофильное образование.\nЗнание C# и платформы .NET.\nПонимание платформы Windows (Win32 API, COM, .NET).\nОпыт использования test-driven development.\nОпыт участия в open source проектах.\nРаботая в нашей компании, вы получите возможность участвовать в разработке высоконагруженного серверного программного обеспечения на языке C/C++. Это действительно интересная и сложная работа: многопоточность, оптимизация, эффективное управление памятью, репликация данных.\n\nОфис компании в Санкт-Петербурге работает с 2010 года (5 минут от Василеостровской и Спортивной). Оформление в полном соответствии с ТК РФ, ДМС, оплачиваемые больничные без справок, эргономичные рабочие места и всё остальное на уровне лучших софтверных компаний Санкт-Петербурга. В том числе для студентов есть возможность совмещать работу и учебу.\n\nМы являемся аккредитованной IT-компанией."

results = recognizer(example)

for result in results:
    print(result['entity'], result['word'], result['score'])

B-ProgrammingLanguage С 0.7995157
B-ProgrammingLanguage C 0.8962386
I-ProgrammingLanguage + 0.92040056
I-ProgrammingLanguage + 0.94713116
I-ProgrammingLanguage , 0.91484946
B-ProgrammingLanguage C 0.893112
I-ProgrammingLanguage # 0.9040786
B-ProgrammingLanguage Java 0.6895581
B-Method структуры 0.72628856
I-Method данных 0.6073363
B-Method алгоритмы 0.6506865
B-ProgrammingLanguage C 0.8920682
B-ProgrammingLanguage C 0.9068745
I-ProgrammingLanguage + 0.93033403
I-ProgrammingLanguage + 0.95131713
I-ProgrammingLanguage . 0.6537125
B-SoftSkills Технический 0.6083414
I-SoftSkills английский 0.67377084
B-ProgrammingLanguage C 0.93546665
I-ProgrammingLanguage # 0.9146142
I-Technology NET 0.34864986
B-Technology Win 0.61565197
B-Technology ##32 0.70128167
I-Technology API 0.37187937
B-Technology COM 0.67728555
B-Technology . 0.2696967
I-Technology NET 0.3350018
B-Method test 0.40669551
I-Technology - 0.35351527
I-Method dr 0.321776
I-Method ##iven 0.35528922
I-Method de 0.38181624
I-Method ##v

Тут уже результаты интересные, я считаю это круто! Сильно бросается в глаза, что все токены языка программирования помечаются лишь тегом "B-". Это правильно, т.к. при обработке языки по типу C++ воспринимались как единое слово.

#### 4.3. Заключение

Заключение.<br>
Результат меня очень впечатлил, я рассчитывал, что модель не сможет хорошо работать на произвольных текстах. Это мой первый опыт работы с Hugging Face, так что, возможно, я не учел некоторые возможности библиотеки и дополнительные варианты улучшения обучаемости модели. <br>
Наивысший результат F1 меры для доп. файликов составил примерно 0.25. Объяснить результат можно следующим: <br>
1) Небольшой размер пакета при обучении (32); <br>
2) Текста, у которых число токенов больше 512, подавались для текста модели пачками по 512 токенов.; <br>
3) Небольшое число эпох. Каждая эпоха занимала около 15 минут на Goolge Colab T4. <br>
4) Наличие больших текстов, из-за чего, вероятно, модели требуется гораздо больше примеров, чтобы выучить все закономерности

Возможно, модель могла бы улучшить свои результаты, если бы я использовал другую архитектуру или текста подавались небольшими осмысленными абзацами

## 5. Код для использования этой модели

Если модель была загружена к вам на компьютер и вы хотите использовать ее в дальнейшем, то вот код для этих целей

Импорт модели и токенайзера:

In [56]:
path_to_model = 'ner_model'

# load our model
model = AutoModelForTokenClassification.from_pretrained(path_to_model)

# load our tokenizer
tokenizer_name = 'KoichiYasuoka/bert-base-russian-upos'
tokenizer = BertTokenizerFast.from_pretrained(tokenizer_name)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Дублирую все необходимые функции:

In [57]:
def text_to_tokens(text):
    pre_tokenizer = Whitespace()
    return [sample[0] for sample in pre_tokenizer.pre_tokenize_str(text)]

# =====================================================================
# =====================================================================

def merge_spitted_tokens(prediction):
    tokens = prediction[0]
    labels = prediction[1]

    new_tokens = []
    new_labels = []

    current_label = labels[0]
    current_token = tokens[0]
    for i in range(len(tokens) - 1):
        token = tokens[i]
        label = labels[i]

        if tokens[i + 1].startswith("##"):
           current_token += tokens[i + 1][2:]
        else:
            new_tokens.append(current_token)
            new_labels.append(current_label)

            current_label = labels[i + 1]
            current_token = tokens[i + 1]

    if tokens[-1].startswith("##"):
        new_tokens.append(current_token)
        new_labels.append(current_label)
    else:
        new_tokens.append(tokens[-1])
        new_labels.append(labels[-1])

    return [new_tokens, new_labels]

# =====================================================================
# =====================================================================

def sinchronize_prediction_by_tokens(prediction, tokens):
    pred_tokens = prediction[0]
    pred_labels = prediction[1]
    new_tokens = []
    new_labels = []

    ind_real = ind_pred = 0
    while (ind_real < len(tokens)) and (ind_pred < len(pred_tokens)):
        real_token = tokens[ind_real]
        pred_token = pred_tokens[ind_pred]
        pred_label = pred_labels[ind_pred]

        if real_token == pred_token:
            new_tokens.append(real_token)
            new_labels.append(pred_label)
            ind_real += 1
            ind_pred += 1
        else:
            new_tokens.append(real_token)
            new_labels.append("O") # assume that splitted labels are not even words
            ind_real += 1
            ind_pred += 1

            if ind_real < len(tokens):
                while (ind_pred < len(pred_tokens)) and (not tokens[ind_real].startswith(pred_tokens[ind_pred])):
                    ind_pred += 1

    return [new_tokens, new_labels]

# =====================================================================
# =====================================================================

def get_predictions(model, tokenzier, df):
    preds = [ [[], []] for i in range(df.shape[0]) ]
    model_id2label = model.config.id2label

    for i in range(df.shape[0]):
        start_index = 0
        current_prediction_full = [[], []]
        df_tokens = df["tokens"][i]

        while True:

            df_tokens_current = df["tokens"][i][start_index:]
            tokenized_input = tokenizer(df_tokens_current, is_split_into_words=True,
                                        padding=True, truncation=True, max_length=512,
                                        add_special_tokens=False, return_tensors="pt")
            model.to('cuda')
            with torch.no_grad():
                outputs = model(tokenized_input["input_ids"].cuda(), tokenized_input["attention_mask"].cuda())

                logits = outputs.logits
                predictions = torch.argmax(logits, dim=2).cuda()
                #label_inds = np.argmax(outputs[0].to('cpu').numpy(), axis=2)

            tokenizer_tokens = tokenizer.convert_ids_to_tokens(tokenized_input['input_ids'][0])
            predicted_labels = [model_id2label[prediction.item()] for prediction in predictions[0]]

            current_prediction = [tokenizer_tokens, predicted_labels]

            current_prediction = merge_spitted_tokens(current_prediction)
            current_prediction = sinchronize_prediction_by_tokens(current_prediction, df_tokens_current)

            current_prediction_full[0] += current_prediction[0]
            current_prediction_full[1] += current_prediction[1]

            start_index = len(current_prediction_full[0])
            if start_index >= len(df_tokens):
                break

        # merge results to one prediction
        preds[i][0] += current_prediction_full[0]
        preds[i][1] += current_prediction_full[1]

        if (i + 1) % 100 == 0:
            print(f"{i + 1} / {df.shape[0]}")

    return preds

In [58]:
example_text = "Требуется программист-стажер на работу в крупной компании. От вас - ответственность + коммуникабельность. Вы лучший!"

tokens = text_to_tokens(example_text)

input = pd.DataFrame({
    'tokens': [tokens]
})

predictions = get_predictions(model, tokenizer, input)

In [59]:
print(np.array(predictions).shape)
print()

for i in range(len(predictions[0][0])):
    print(predictions[0][0][i], predictions[0][1][i])

(1, 2, 20)

Требуется O
программист O
- O
стажер O
на O
работу O
в O
крупной O
компании O
. O
От O
вас O
- O
ответственность B-SoftSkills
+ O
коммуникабельность B-SoftSkills
. O
Вы O
лучший O
! O


Полученные результаты можно дополнительно обработать, чтобы получить наиболее желаемый вид!